# join_2dataframes

In [1]:
%load_ext autoreload
%autoreload 2

# The autoreload extension is already loaded. To reload it, use:
#  %reload_ext autoreload

In [2]:
import os
import sys
import numpy as np
import pandas as pd

pd.options.display.float_format = '{:8,.2f}'.format

In [3]:
# mylib:
my_library = os.path.expanduser('~/.myconfigs')
my_spark = os.path.expanduser('~/spark2_dfanalysis')
sys.path.append(my_library)
sys.path.append(my_spark)

In [4]:
import pyspark as spark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# spark = SparkSession.builder.appName('myappname').getOrCreate()
# print(spark)

In [5]:
# print(dir(pyspark))
# print(dir(pyspark.sql))
# print(dir(pyspark.rdd))

In [6]:
from shared.app_context import *

ctx = ApplicationContext("Dev-Job")
print(ctx.spark)

In [7]:
# print(sys.path)
from builder.DataFrameBuild import *

## DataFrameBuild Class:
### x = DataFrameBuild(ctx.spark)

In [8]:
x = DataFrameBuild(ctx.spark)

DF1

In [9]:
x = DataFrameBuild(ctx.spark)
num = 500
df1 = x.arrays_to_dataframe([[int(x) for x in np.linspace(1,num,num)],
                             x.build_array("string",num=num,width=8),
                             x.build_array("integer",num=num,nrange=(1,4)),
                             x.build_array("integer",num=num,nrange=(1,12)),
                             x.build_array("double",num=num,nrange=(0.0,10000))],
                            ['index','passwords','quarter','month','price'])

DF2

In [10]:
x = DataFrameBuild(ctx.spark)
num = 500
df2 = x.arrays_to_dataframe([[int(x) for x in np.linspace(1,num,num)],
                             x.build_array("string",num=num,width=8),
                             x.build_array("integer",num=num,nrange=(1,4)),
                             x.build_array("integer",num=num,nrange=(1,12)),
                             x.build_array("double",num=num,nrange=(0.0,10000))],
                             ['index','passwords','quarter','month','price'])

In [11]:
df1.limit(10).show()

+-----+---------+-------+-----+------------------+
|index|passwords|quarter|month|             price|
+-----+---------+-------+-----+------------------+
|    1| huivhhmu|      1|    3|2630.0502273448824|
|    2| yeqeeuln|      2|    3| 8607.493101038992|
|    3| tczhbial|      3|    9| 7319.622097388472|
|    4| psohunad|      2|    5|1051.9242606094658|
|    5| bziyoyng|      1|    7|  9467.83973201856|
|    6| fochpsqc|      3|    9| 8977.763785809775|
|    7| xoqvwfof|      3|    2| 8164.109542146126|
|    8| lmykhwfr|      2|    3|5831.6693574204455|
|    9| fvccucob|      4|    7|  76.6504561997905|
|   10| lobirzbj|      4|    5|  3007.90576308791|
+-----+---------+-------+-----+------------------+



In [12]:
df2.limit(10).show()

+-----+---------+-------+-----+------------------+
|index|passwords|quarter|month|             price|
+-----+---------+-------+-----+------------------+
|    1| mesobjzv|      4|    2|4473.7634872110475|
|    2| ehdalvrh|      1|    3| 2598.552149475184|
|    3| rgwpselg|      1|    4| 7419.241704989037|
|    4| etizuslt|      4|    4| 9787.726663196883|
|    5| hatwuigy|      2|    9|7896.8214705474575|
|    6| pmxiqmsl|      4|    2|1810.1809488475496|
|    7| pqmepioc|      1|    2| 946.3025831009786|
|    8| afbmkhco|      1|    1| 1027.703719523463|
|    9| dliymgco|      4|    9| 6265.596032886944|
|   10| wdvbjjdo|      3|   10| 6911.999954002019|
+-----+---------+-------+-----+------------------+



## Join df1 and df2 on indices.

In [60]:
df_index = df1.join(df2,df1.index == df2.index)
df_index = df1.join(df2,df1["index"] == df2["index"])

In [61]:
%%time
df_index.cache()

Wall time: 2 ms


DataFrame[index: bigint, passwords: string, quarter: bigint, month: bigint, price: double, index: bigint, passwords: string, quarter: bigint, month: bigint, price: double]

In [62]:
df_index.show(20)

+-----+---------+-------+-----+------------------+-----+---------+-------+-----+------------------+
|index|passwords|quarter|month|             price|index|passwords|quarter|month|             price|
+-----+---------+-------+-----+------------------+-----+---------+-------+-----+------------------+
|   26| qerglufq|      2|    1|3030.4061156760918|   26| wxgpatxj|      1|    2|3387.6597958352263|
|   29| aigyphwn|      1|   10|  6252.80216631731|   29| oiemkyzy|      2|    1|4023.8255056872863|
|  474| udgmapre|      2|    4| 9749.675099684515|  474| rjowkrqh|      4|    9| 6632.458760756636|
|   65| bhlxckok|      3|    9| 7814.899323398169|   65| oygsbzid|      2|    3|3433.3381516495733|
|  191| szbxvtnz|      4|    9| 9491.417542858147|  191| osvytxei|      2|    9|1161.2357048689792|
|  418| pnpfiwrk|      3|   10| 7230.036629785022|  418| vaumhtcy|      4|    8|  36.0142648019568|
|  222| ccymeaxh|      1|    9|  5908.93062973059|  222| gcomxhhv|      3|    5| 8488.318332444042|


In [58]:
df_index.columns

['index',
 'passwords',
 'quarter',
 'month',
 'price',
 'index',
 'passwords',
 'quarter',
 'month',
 'price']

## Join df1 and df2 on indices. (round 2)

In [71]:
df_i2 = df1.alias("o")\
.join(df2.alias("t"),col("o.index") == col("t.index"))

# df_index = df1.join(df2,df1["index"] == df2["index"])

In [72]:
%%time
df_i2.cache()

Wall time: 0 ns


DataFrame[index: bigint, passwords: string, quarter: bigint, month: bigint, price: double, index: bigint, passwords: string, quarter: bigint, month: bigint, price: double]

In [73]:
df_i2.show(20)

+-----+---------+-------+-----+------------------+-----+---------+-------+-----+------------------+
|index|passwords|quarter|month|             price|index|passwords|quarter|month|             price|
+-----+---------+-------+-----+------------------+-----+---------+-------+-----+------------------+
|   26| qerglufq|      2|    1|3030.4061156760918|   26| wxgpatxj|      1|    2|3387.6597958352263|
|   29| aigyphwn|      1|   10|  6252.80216631731|   29| oiemkyzy|      2|    1|4023.8255056872863|
|  474| udgmapre|      2|    4| 9749.675099684515|  474| rjowkrqh|      4|    9| 6632.458760756636|
|   65| bhlxckok|      3|    9| 7814.899323398169|   65| oygsbzid|      2|    3|3433.3381516495733|
|  191| szbxvtnz|      4|    9| 9491.417542858147|  191| osvytxei|      2|    9|1161.2357048689792|
|  418| pnpfiwrk|      3|   10| 7230.036629785022|  418| vaumhtcy|      4|    8|  36.0142648019568|
|  222| ccymeaxh|      1|    9|  5908.93062973059|  222| gcomxhhv|      3|    5| 8488.318332444042|


In [74]:
df_i2.columns

['index',
 'passwords',
 'quarter',
 'month',
 'price',
 'index',
 'passwords',
 'quarter',
 'month',
 'price']

## Create a new column calculated from current column values.